In [1]:
import numpy as np

In [2]:
#Hidden Markov Model
#state transition probabilities
transitionProbs = np.array([\
        [0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.5, 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0. , 0.5, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0. , 0.5, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.5, 0. , 0.5, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0.5, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 1. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. ]
                      ])

#HMM observation probabilities given a state
evidenceProbs = np.array([\
        [0.3, 0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.3],
       [0.3, 0.3, 0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.3, 0.3, 0.3, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.3, 0.3, 0.3, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.3, 0.3, 0.3, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.3, 0.3, 0.3, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.3, 0.3, 0.3, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.3, 0.3, 0.3, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.3, 0.3, 0.3],
       [0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.3, 0.3]
                         ])

#HMM initial uniform prior distribution for v1
initialPrior = 0.1*np.ones(10)
#HMM Observed Sequence
ObsSeq = np.array([8,6,4,6,5,4,5,5,7,9])

states = np.array([1,2,3,4,5,6,7,8,9,10])

In [3]:
def HMMvtstates(transitionProbs,evidenceProbs,ObsSeq,initialPrior,states):
    
    vts = np.zeros(len(ObsSeq)).astype(int)
    eps = 1e-8
    logInitial = np.log(initialPrior+eps)
    logEvid = np.log(evidenceProbs+eps)
    logTrans = np.log(transitionProbs+eps)

    Prob = np.zeros((len(ObsSeq),len(ObsSeq)))
    Seq = np.zeros((len(ObsSeq),len(ObsSeq)))
    for k in states:
        #table for probability dists for sequence of observed values
        Prob[k-1,0] = logInitial[k-1] + logEvid[k-1,ObsSeq[0]]
        #bookkeep sequences
        Seq[k-1,0] = 0
        
    #update tables for each time step observation
    for t in states[:-1]:
        Prob[:,t] = np.max(Prob[:,t-1] + logEvid[:,ObsSeq[t]].reshape(10,-1) + logTrans,axis=1)
        Seq[:,t] = np.argmax(Prob[:,t-1] + logEvid + logTrans,axis=1)

    #build most likely sequence from last to first
    vts[-1] = np.argmax(Prob[:,len(ObsSeq)-1])
    for t in range(len(ObsSeq)-1,0,-1):
        vts[t-1] = Seq[vts[t],t]

    return vts.tolist()

In [4]:
print('Observation Sequence: \n', ObsSeq.tolist())
stateSequence = HMMvtstates(transitionProbs,evidenceProbs,ObsSeq,initialPrior,states)
print('Most Likely Sequence of v_t values: \n',stateSequence)

Observation Sequence: 
 [8, 6, 4, 6, 5, 4, 5, 5, 7, 9]
Most Likely Sequence of v_t values: 
 [7, 6, 5, 6, 5, 4, 5, 6, 7, 8]
